In [19]:
import glob
import os

import pandas as pd
import numpy as np

## MTurk

In [20]:
mturk_fpaths = glob.glob("../results/mturk/pilot*.csv")
mturk_fpaths

['../results/mturk\\pilot01_Workplace_Survey_1Page_1.csv',
 '../results/mturk\\pilot02_Workplace_Survey_1.csv',
 '../results/mturk\\pilot03_Batch_4542715_batch_results.csv',
 '../results/mturk\\pilot04_Workplace_Survey_Pilot_815_2.csv',
 '../results/mturk\\pilot05_Workplace_Survey_Pilot_815_3.csv',
 '../results/mturk\\pilot06_Workplace_Survey_Pilot_810_12.csv',
 '../results/mturk\\pilot07_Workplace_Survey_Pilot_810_11.csv',
 '../results/mturk\\pilot08_Workplace_Survey_Pilot_810_10.csv',
 '../results/mturk\\pilot09_Workplace_Survey_Pilot_810_8.csv',
 '../results/mturk\\pilot10_Workplace_Survey_Pilot_810_7.csv',
 '../results/mturk\\pilot11_Workplace_Survey_Pilot_810_6.csv',
 '../results/mturk\\pilot12_Workplace_Survey_Pilot_810_5.csv',
 '../results/mturk\\pilot13_Workplace_Survey_Pilot_810_4.csv',
 '../results/mturk\\pilot14_Workplace_Survey_Pilot_87_8.csv',
 '../results/mturk\\pilot15_Workplace_Survey_Pilot_87_6.csv',
 '../results/mturk\\pilot16_Workplace_Survey_Pilot_87_4.csv',
 '../re

In [21]:
all_dfs = []
for cur_mturk_fpath in mturk_fpaths:
    cur_fname = os.path.basename(cur_mturk_fpath)
    cur_pnum = int(cur_fname[5:7])
    #print(cur_pnum)
    cur_df = pd.read_csv(cur_mturk_fpath)
    #display(cur_df)
    cur_df['source'] = os.path.basename(cur_mturk_fpath)
    # Rename some cols for later
    cur_df.rename(columns={'WorkerId':'worker_id'}, inplace=True)
    cur_df['accepted_offer'] = False
    # Replace accepted=False if there's no survey code
    # Check the name of the survey code var
    scode_var = 'Answer.surveyCode'
    if scode_var in cur_df.columns:
        cur_df['mturk_code'] = cur_df[scode_var]
    if scode_var not in cur_df.columns:
        # Try
        scode_var = 'Answer.surveycode'
        cur_df['mturk_code'] = cur_df[scode_var]
    cur_df.at[cur_df[scode_var] != "{}", 'accepted_offer'] = True
    cur_df['mturk_code'] = cur_df['mturk_code'].apply(lambda x: str(x))
    # And also a few people put stuff after the code, so delete that
    cur_df['mturk_code'] = cur_df['mturk_code'].apply(lambda x: x[:8] if len(x) > 8 else x)
    all_dfs.append(cur_df)

In [22]:
mturk_df = pd.concat(all_dfs)

In [23]:
mturk_df.reset_index(drop=True, inplace=True)

In [24]:
mturk_df.columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'worker_id', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.hit_id',
       'Input.offer_amt', 'Input.offer_hash', 'Answer.age',
       'Answer.hiddenhitid', 'Answer.onlinehrs', 'Answer.reason',
       'Answer.surveycode', 'Approve', 'Reject', 'source', 'accepted_offer',
       'mturk_code', 'Answer.acceptOffer', 'Answer.surveyCode'],
      dtype='object')

In [25]:
len(mturk_df)

217

In [26]:
mturk_df[pd.isna(mturk_df['mturk_code'])]

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,Answer.onlinehrs,Answer.reason,Answer.surveycode,Approve,Reject,source,accepted_offer,mturk_code,Answer.acceptOffer,Answer.surveyCode


In [27]:
mturk_df.to_csv("../results/mturk_clean.csv", index=False)

In [28]:
# But now we need to (a) get the offer amounts for people who declined, and
# (b) merge in the Qualtrics data for people who accepted

## RestDB

The idea here is: *if* the MTurk id from the MTurk results df is in the DB, then just get the offer amt from there. Otherwise, get it from the qualtrics data

In [29]:
restdb_fpath = os.path.join("..","results","restdb","restdb_2021-09-03.csv")

In [30]:
restdb_df = pd.read_csv(restdb_fpath)

In [31]:
# Drop the "[Functions]" row
restdb_df = restdb_df[restdb_df['restdb_id'] != "[Functions]"].copy()

In [32]:
restdb_df['restdb_id'] = pd.to_numeric(restdb_df['restdb_id'])

In [33]:
len(restdb_df)

313

In [34]:
restdb_df.to_csv("../results/restdb_clean.csv", index=False)

## First merge: mturk data with restdb data

In [35]:
# Now merge into mturk df
# (old)
#mturk_df['rest"db_wage'] = mturk_df['WorkerId'].apply(lambda x: wage_map[x] if x in wage_map else None)
# (new)
rdb_merged_df = restdb_df.merge(mturk_df, on=['worker_id'], how='outer', indicator="rdb_mturk_merge",
                                suffixes=['_restdb','_mturk'])

In [36]:
rdb_merged_df['rdb_mturk_merge'].value_counts()

both          194
left_only     125
right_only     25
Name: rdb_mturk_merge, dtype: int64

In [37]:
len(rdb_merged_df)

344

In [38]:
rdb_merged_df['response_case'] = "mt_submitter"
rdb_merged_df.at[rdb_merged_df['rdb_mturk_merge'] == "left_only","response_case"] = "mt_returner"
rdb_merged_df.at[rdb_merged_df['rdb_mturk_merge'] == "right_only","response_case"] = "no_restdb"

In [39]:
# Code for picking the final offer, now done at end
"""
mturk_df['offer'] = None
for cur_index in mturk_df.index:
    final_wage = ""
    cur_input = mturk_df.loc[cur_index,'Input.offer_amt']
    if not pd.isna(cur_input):
        final_wage = cur_input
    else:
        cur_restdb = mturk_df.loc[cur_index,'restdb_wage']
        final_wage = cur_restdb
    mturk_df.at[cur_index,'offer'] = final_wage
"""

'\nmturk_df[\'offer\'] = None\nfor cur_index in mturk_df.index:\n    final_wage = ""\n    cur_input = mturk_df.loc[cur_index,\'Input.offer_amt\']\n    if not pd.isna(cur_input):\n        final_wage = cur_input\n    else:\n        cur_restdb = mturk_df.loc[cur_index,\'restdb_wage\']\n        final_wage = cur_restdb\n    mturk_df.at[cur_index,\'offer\'] = final_wage\n'

In [40]:
# How many still need offer?
#no_offer = mturk_df[pd.isna(mturk_df['offer'])].copy()

## Parse Qualtrics data

In [94]:
# Cool, the remaining ones just need the offer merged in from Qualtrics

In [95]:
qualtrics_fpaths = glob.glob("../results/qualtrics/*.xlsx")

In [96]:
qualtrics_fpaths

['../results/qualtrics\\Job+Quality+NonWM+2.0_August+24,+2021_14.40.xlsx',
 '../results/qualtrics\\Job+Quality+NonWM+3.0_August+24,+2021_14.41.xlsx',
 '../results/qualtrics\\Job+Quality+NonWM+4.0_August+24,+2021_14.41.xlsx',
 '../results/qualtrics\\Job+Quality+NonWM+5.0_August+24,+2021_14.42.xlsx',
 '../results/qualtrics\\Job+Quality+NonWM+7.0_August+24,+2021_14.43.xlsx',
 '../results/qualtrics\\Job+Quality+NonWM+8.0_August+24,+2021_14.44.xlsx',
 '../results/qualtrics\\Job+Quality+NonWM+9.0_August+30,+2021_15.45.xlsx']

In [97]:
q_map = {
    'Q78': 'race',
    'Q78_6_TEXT': 'race_6_TEXT',
    'State': 'state',
    'Q79': 'gender',
    'Q79_4_TEXT': 'gender_4_TEXT',
    'Q176': 'education',
    'Q42': 'currently_employed',
    'Q158': 'fulltime',
    'Q47': 'most_recent_job',
    'Q159': 'most_recent_fulltime',
    'Q139': 'weeks_unemployment',
    'Q81': 'how_easy',
    'Q174': 'lowest_wage',
    'Q175': 'savings',
    'Q10': 'response1',
    'Q40': 'what_would_it_take',
    'Q155': 'response2',
    'Q158.1': 'response3',
    'Q161': 'response4',
    'Q164': 'response5',
    'Q167': 'response6',
    'Q170': 'response7',
    'Q176.1': 'response8',
    'Q179': 'response9',
    'Q182': 'response10',
    'Q185': 'response11',
    'Q25': 'jobtitle'
}
extra_q_map = {
    'Q207': 'response17',
    'Q210': 'response18',
    'Q213': 'response19',
    'Q216': 'response20',
}
#q_drop = ['Q11','Q22','Q12','Q90','Q13','Q14','Q15','Q20','Q19','Q16','Q17','Q18']

In [98]:
# First combine all the qualtrics files into one big file
all_qual_dfs = []
for cur_fpath in qualtrics_fpaths:
    qualtrics_fname = os.path.basename(cur_fpath)
    print(f"Loading {cur_fpath}")
    if "2.0" in cur_fpath:
        print("Version 2.0!")
    cur_df = pd.read_excel(cur_fpath)
    # First row is just labels
    cur_df = cur_df.iloc[1:].copy()
    print(f"Num obs: {len(cur_df)}")
    cur_df.rename(columns={'offer_amt':'qualtrics_offer_amt','offer':'qualtrics_offer'}, inplace=True)
    # Finally, we need to rename the "QXXX" questions from the early versions
    if "Q78" in cur_df.columns:
        cur_df.rename(columns=q_map, inplace=True)
        #cur_df.drop(columns=q_drop, inplace=True)
    if "Q207" in cur_df.columns:
        cur_df.rename(columns=extra_q_map, inplace=True)
        #cur_df.drop(columns=q_drop, inplace=True, errors='ignore')
    cur_df['source_qualtrics'] = qualtrics_fname
    #break
    all_qual_dfs.append(cur_df)

Loading ../results/qualtrics\Job+Quality+NonWM+2.0_August+24,+2021_14.40.xlsx
Version 2.0!
Num obs: 8
Loading ../results/qualtrics\Job+Quality+NonWM+3.0_August+24,+2021_14.41.xlsx


c:\users\jpjac\appdata\local\programs\python\python39\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Num obs: 13
Loading ../results/qualtrics\Job+Quality+NonWM+4.0_August+24,+2021_14.41.xlsx
Num obs: 3
Loading ../results/qualtrics\Job+Quality+NonWM+5.0_August+24,+2021_14.42.xlsx
Num obs: 53
Loading ../results/qualtrics\Job+Quality+NonWM+7.0_August+24,+2021_14.43.xlsx
Num obs: 3
Loading ../results/qualtrics\Job+Quality+NonWM+8.0_August+24,+2021_14.44.xlsx
Num obs: 42
Loading ../results/qualtrics\Job+Quality+NonWM+9.0_August+30,+2021_15.45.xlsx
Num obs: 88


In [99]:
with pd.option_context('display.max_columns', None):
    display(cur_df)

StartDate              EndDate      Status       IPAddress  \
1   2021-08-28 20:27:00  2021-08-28 20:28:41  IP Address  73.162.172.111   
2   2021-08-28 20:35:26  2021-08-28 20:44:03  IP Address   196.17.64.193   
3   2021-08-28 20:36:07  2021-08-28 20:44:50  IP Address    63.75.243.75   
4   2021-08-28 20:36:50  2021-08-28 20:45:06  IP Address   63.75.247.128   
5   2021-08-28 20:35:22  2021-08-28 20:45:30  IP Address   71.235.206.17   
..                  ...                  ...         ...             ...   
84  2021-08-28 22:40:19  2021-08-28 22:48:53  IP Address     67.2.60.107   
85  2021-08-28 22:22:46  2021-08-28 22:48:55  IP Address  160.202.94.160   
86  2021-08-28 23:37:43  2021-08-29 00:36:14  IP Address   12.198.49.145   
87  2021-08-28 23:54:27  2021-08-29 00:38:29  IP Address   12.198.49.142   
88  2021-08-28 23:43:49  2021-08-29 00:45:09  IP Address    196.17.67.95   

   Progress Duration (in seconds) Finished                RecordedDate  \
1     100.0                 101.0     True  2021-08-28 20:28:42.970000   
2     100.0                 516.0     True  2021-08-28 20:44:05.062000   
3     100.0                 522.0     True  2021-08-28 20:44:51.693000   
4     100.0                 496.0     True  2021-08-28 20:45:07.273000   
5     100.0                 608.0     True  2021-08-28 20:45:31.433000   
..      ...                   ...      ...                         ...   
84     99.0                 513.0    False  2021-08-29 22:48:55.607000   
85     99.0                1568.0    False  2021-08-29 22:48:57.969000   
86     99.0                3510.0    False  2021-08-30 00:36:15.906000   
87     26.0                2642.0    False  2021-08-30 00:38:30.161000   
88     96.0                3680.0    False  2021-08-30 00:45:10.661000   

           ResponseId RecipientLastName RecipientFirstName RecipientEmail  \
1   R_2SCcccBFax9F15U               NaN                NaN            NaN   
2   R_1rHNSBs7eSg4jss               NaN                NaN            NaN   
3   R_XYv1laaih1cEu1r               NaN                NaN            NaN   
4   R_3nDH4LoPfzrWHkc               NaN                NaN            NaN   
5   R_8nWrGPCIPaNb1df               NaN                NaN            NaN   
..                ...               ...                ...            ...   
84  R_339k0ZgqbjKHaxg               NaN                NaN            NaN   
85  R_W0t0k9avelWLA9H               NaN                NaN            NaN   
86  R_3fq6QwKYRLuBA17               NaN                NaN            NaN   
87  R_2aWZB1koLRpbjOA               NaN                NaN            NaN   
88  R_2AWR0j3dHEu5sK6               NaN                NaN            NaN   

   ExternalReference LocationLatitude LocationLongitude DistributionChannel  \
1                NaN        37.790604       -122.241196           anonymous   
2                NaN        34.054398       -118.244003           anonymous   
3                NaN        37.751007        -97.821999           anonymous   
4                NaN        37.751007        -97.821999           anonymous   
5                NaN        41.748993        -72.690399           anonymous   
..               ...              ...               ...                 ...   
84               NaN              NaN               NaN           anonymous   
85               NaN              NaN               NaN           anonymous   
86               NaN              NaN               NaN           anonymous   
87               NaN              NaN               NaN           anonymous   
88               NaN              NaN               NaN           anonymous   

   UserLanguage           state                       race race_6_TEXT  \
1            EN         Arizona                      White         NaN   
2            EN         Georgia                      White         NaN   
3            EN         Florida  Asian or Pacific Islander         NaN   
4            EN      California           

In [100]:
full_qual_df = pd.concat(all_qual_dfs)

In [101]:
with pd.option_context('display.max_columns',None):
    display(full_qual_df[full_qual_df['source_qualtrics'].str.contains("2.0")])

StartDate              EndDate      Status        IPAddress  \
1  2021-07-17 17:45:31  2021-07-17 17:54:05  IP Address   68.160.235.238   
2  2021-07-17 17:57:20  2021-07-17 17:59:28  IP Address   64.121.102.112   
3  2021-07-17 17:45:36  2021-07-17 18:00:14  IP Address       66.56.8.16   
4  2021-07-17 17:51:52  2021-07-17 18:00:47  IP Address    73.159.73.173   
5  2021-07-17 18:11:54  2021-07-17 18:19:12  IP Address  163.123.221.198   
6  2021-07-17 18:31:25  2021-07-17 18:36:18  IP Address   173.24.227.204   
7  2021-07-17 19:45:35  2021-07-17 19:51:14  IP Address    73.67.133.254   
8  2021-07-17 18:03:21  2021-07-17 18:04:03  IP Address   67.172.249.152   

  Progress Duration (in seconds) Finished                RecordedDate  \
1    100.0                 513.0     True  2021-07-17 17:54:06.250000   
2    100.0                 127.0     True  2021-07-17 17:59:28.541000   
3    100.0                 878.0     True  2021-07-17 18:00:15.355000   
4    100.0                 535.0     True  2021-07-17 18:00:48.385000   
5    100.0                 438.0     True  2021-07-17 18:19:13.183000   
6    100.0                 292.0     True  2021-07-17 18:36:19.122000   
7    100.0                 339.0     True  2021-07-17 19:51:15.278000   
8     23.0                  41.0    False  2021-07-18 18:04:07.069000   

          ResponseId RecipientLastName RecipientFirstName RecipientEmail  \
1  R_1fmaehpl5yBIW6k               NaN                NaN            NaN   
2  R_1IgRWDYxcjdFQWy               NaN                NaN            NaN   
3  R_2ttDtpF3Y87wARn               NaN                NaN            NaN   
4  R_3kvOAqEFdsVaWEr               NaN                NaN            NaN   
5  R_2VeE3YBGhB1YUnH               NaN                NaN            NaN   
6  R_1Q0MPGHDKfseV5J               NaN                NaN            NaN   
7  R_ZEkPvKvYuTdLllf               NaN                NaN            NaN   
8  R_23WeydR7erPeyA6               NaN                NaN            NaN   

  ExternalReference LocationLatitude LocationLongitude DistributionChannel  \
1               NaN        40.738007        -73.985802           anonymous   
2               NaN        40.588501        -75.464203           anonymous   
3               NaN        33.848602        -84.355698           anonymous   
4               NaN        42.691498        -71.157898           anonymous   
5               NaN        37.751007        -97.821999           anonymous   
6               NaN        37.167999        -93.303902           anonymous   
7               NaN        45.503296       -122.774399           anonymous   
8               NaN              NaN               NaN           anonymous   

  UserLanguage          state                       race race_6_TEXT  gender  \
1           EN       New York  Asian or Pacific Islander         NaN    Male   
2           EN   Pennsylvania                      White         NaN    Male   
3           EN        Georgia  Asian or Pacific Islander         NaN    Male   
4           EN  Massachusetts        Latino/a (any race)         NaN    Male   
5           EN        Florida                      White         NaN    Male   
6           EN       Missouri                      White         NaN    Male   
7           EN         Oregon                      White         NaN    Male   
8           EN           Utah                      White         NaN  Female   

  gender_4_TEXT                   education currently_employed   fulltime  \
1           NaN           Bachelor's degree                Yes  Full-time   
2           NaN           Bachelor's degree                Yes  Full-time   
3           NaN             Master's degree                Yes  Full-time   
4           NaN           Bachelor's degree                Yes  Full-time   
5           NaN           Bachelor's degree                 No        NaN   
6           NaN  Some college but no degree                Yes  Full-time   
7           NaN  High 

In [102]:
def drop_col(c):
        return "cur_" in c or "name_" in c or "val_" in c or "tense" in c
# Drop extraneous vars
cols_to_drop = [c for c in full_qual_df.columns if drop_col(c)]
full_qual_df.drop(columns=cols_to_drop, inplace=True)
# Rename so we know these are the offers from *within qualtrics*
full_qual_df['mturk_code'] = full_qual_df['mturk_code'].apply(lambda x: str(x))
# Finally, can replace qualtrics_offer_amt = qualtrics_offer if qualtrics_offer_amt is NaN
#for row_index, cur_row in full_qual_df.iterrows():
#    cur_amt = cur_row['qualtrics_offer_amt']
#    cur_source = cur_row['source_qualtrics']
#    #print(cur_source)
#    #print(cur_amt)
#    if pd.isna(cur_amt):
#        # Fill with qualtrics_offer
#        print("Replacing")
#        full_qual_df.at[row_index, 'qualtrics_offer_amt'] = full_qual_df.loc[row_index, 'qualtrics_offer']
# And now we can drop qualtrics_offer, to avoid confusion
#full_qual_df.drop(columns=['qualtrics_offer'], inplace=True)

In [103]:
len(full_qual_df)

210

In [104]:
full_qual_df[['qualtrics_offer_amt','source_qualtrics']]

,qualtrics_offer_amt,source_qualtrics
1,$1.01,"Job+Quality+NonWM+2.0_August+24,+2021_14.40.xlsx"
2,$0.90,"Job+Quality+NonWM+2.0_August+24,+2021_14.40.xlsx"
3,$0.50,"Job+Quality+NonWM+2.0_August+24,+2021_14.40.xlsx"
4,$0.95,"Job+Quality+NonWM+2.0_August+24,+2021_14.40.xlsx"
5,$0.99,"Job+Quality+NonWM+2.0_August+24,+2021_14.40.xlsx"
...,...,...
84,NaN,"Job+Quality+NonWM+9.0_August+30,+2021_15.45.xlsx"
85,NaN,"Job+Quality+NonWM+9.0_August+30,+2021_15.45.xlsx"
86,NaN,"Job+Quality+NonWM+9.0_August+30,+2021_15.45.xlsx"
87,NaN,"Job+Quality+NonWM+9.0_August+30,+2021_15.45.xlsx"


In [105]:
#with pd.option_context('display.max_rows', 1000):
#    display(full_qual_df[['qualtrics_offer','qualtrics_offer_amt']])

In [106]:
full_qual_df['mt_id']

1                NaN
2                NaN
3                NaN
4                NaN
5                NaN
           ...      
84    A1607HOEF37WK7
85    A1PAKREEY798TM
86    A12DBZUCGKNBKA
87     A4VX0HIO63OL1
88    A2MXVF2HKV7CFI
Name: mt_id, Length: 210, dtype: object

In [107]:
full_qual_df['qualtrics_offer_amt']

1     $1.01
2     $0.90
3     $0.50
4     $0.95
5     $0.99
      ...  
84      NaN
85      NaN
86      NaN
87      NaN
88      NaN
Name: qualtrics_offer_amt, Length: 210, dtype: object

In [108]:
full_qual_df.to_csv("../results/qualtrics_clean.csv", index=False)

In [109]:
with pd.option_context('display.max_columns',None):
    display(full_qual_df[full_qual_df['mturk_code'] == "87087738"])

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,state,race,race_6_TEXT,gender,gender_4_TEXT,education,currently_employed,fulltime,most_recent_job,most_recent_fulltime,weeks_unemployment,how_easy,lowest_wage,savings,jobtitle,Q11,Q22,Q12,Q90,Q13,Q14,Q15,Q20,Q19,Q16,Q17,Q18,response1,what_would_it_take,response2,response3,response4,response5,response6,response7,response8,response9,response10,response11,mturk_code,additional_tasks,ever_employed,qualtrics_offer_amt,end_message,entered_jobtitle,entered_wage,entered_hrs,entered_controlhrs,entered_friends,entered_commute,entered_physical,entered_coworkers,entered_suprespect,entered_supfair,entered_express,entered_skills,entered_sickleave,coin_flip,refuseoffer,source_qualtrics,qualtrics_offer,offer_case,wage,hrs,controlhrs,sickleave,friends,commute,physical,skills,vaccine,express,coworkers,suprespect,supfair,response12,response13,response14,response15,response16,num_tasks,mt_id,entered_vaccine,response17,response18,response19,response20,end_essay,response21,response22,response23,response24,response25,response26,response27,response28,response29,response30,generated_wage_01,generated_hrsweek_01,generated_controlhrs_01,generated_paidsick_01,generated_friends_01,generated_commute_01,generated_physical_01,generated_skills_01,generated_vaccine_01,generated_express_01,generated_coworkers_01,generated_suprespect_01,generated_supfair_01,generated_wage_02,generated_hrsweek_02,generated_controlhrs_02,generated_paidsick_02,generated_friends_02,generated_commute_02,generated_physical_02,generated_skills_02,generated_vaccine_02,generated_express_02,generated_coworkers_02,generated_suprespect_02,generated_supfair_02,generated_wage_03,generated_hrsweek_03,generated_controlhrs_03,generated_paidsick_03,generated_friends_03,generated_commute_03,generated_physical_03,generated_skills_03,generated_vaccine_03,generated_express_03,generated_coworkers_03,generated_suprespect_03,generated_supfair_03,generated_wage_04,generated_hrsweek_04,generated_controlhrs_04,generated_paidsick_04,generated_friends_04,generated_commute_04,generated_physical_04,generated_skills_04,generated_vaccine_04,generated_express_04,generated_coworkers_04,generated_suprespect_04,generated_supfair_04,generated_wage_05,generated_hrsweek_05,generated_controlhrs_05,generated_paidsick_05,generated_friends_05,generated_commute_05,generated_physical_05,generated_skills_05,generated_vaccine_05,generated_express_05,generated_coworkers_05,generated_suprespect_05,generated_supfair_05,generated_wage_06,generated_hrsweek_06,generated_controlhrs_06,generated_paidsick_06,generated_friends_06,generated_commute_06,generated_physical_06,generated_skills_06,generated_vaccine_06,generated_express_06,generated_coworkers_06,generated_suprespect_06,generated_supfair_06,generated_wage_07,generated_hrsweek_07,generated_controlhrs_07,generated_paidsick_07,generated_friends_07,generated_commute_07,generated_physical_07,generated_skills_07,generated_vaccine_07,generated_express_07,generated_coworkers_07,generated_suprespect_07,generated_supfair_07,generated_wage_08,generated_hrsweek_08,generated_controlhrs_08,generated_paidsick_08,generated_friends_08,generated_commute_08,generated_physical_08,generated_skills_08,generated_vaccine_08,generated_express_08,generated_coworkers_08,generated_suprespect_08,generated_supfair_08,generated_wage_09,generated_hrsweek_09,generated_controlhrs_09,generated_paidsick_09,generated_friends_09,generated_commute_09,generated_physical_09,generated_skills_09,generated_vaccine_09,generated_express_09,generated_coworkers_09,generated_suprespect_09,generated_supfair_09,generated_wage_10,generated_hrsweek_10,generated_controlhrs_10,generated_paidsick_10,generated_friends_10,generated_commute_10,generated_physical_10,generated_skills_10,generated_vaccine

## Merging again

In [110]:
merged_df = rdb_merged_df.merge(full_qual_df, on=['mturk_code'], how='left', indicator="qualtrics_merge",
                                suffixes=['_mturk','_qualtrics'])

In [111]:
merged_df['qualtrics_merge'].value_counts()

left_only     192
both          152
right_only      0
Name: qualtrics_merge, dtype: int64

In [112]:
len(merged_df)

344

In [113]:
#qualtrics_only = merged_df[merged_df['qualtrics_merge'] == 'right_only'].copy()

In [114]:
#qualtrics_only['']

In [115]:
merged_df['final_offer'] = None

In [116]:
# Ok, here are all the different offer cols, in order of precedence
# Input.offer_amt, restdb_wage, qualtrics_offer_amt, qualtrics_offer

In [117]:
# Update the final offer to be the qualtrics offer, if we haven't been able
# to get the offer from anywhere else
for row_index, cur_row in merged_df.iterrows():
    input_val = cur_row['Input.offer_amt']
    if not pd.isna(input_val):
        merged_df.at[row_index, 'final_offer'] = input_val
        continue
    # If we got here, Input.offer_amt was None
    restdb_val = cur_row['restdb_wage']
    if not pd.isna(restdb_val):
        merged_df.at[row_index, 'final_offer'] = restdb_val
        continue
    # If here, Input.offer_amt and restdb_wage were None
    qualtrics_val = cur_row['qualtrics_offer_amt']
    if not pd.isna(qualtrics_val):
        merged_df.at[row_index, 'final_offer'] = qualtrics_val
        continue

In [118]:
# These ones, unfortunately, RestDB broke on, so they didn't receive any offer,
# so we have to drop them:
old_len = len(merged_df)
broken_restdb = ['A1JRN43LLN30FQ','A2IC4IMH7G9UKU','A1CAEF74MR3VRU','A2PAD8WUL40WMI',
                 'A5E2P5F1AXYZ1','A23G1L7KYHK9F2','A7EP5KKL8YQPX',
                 'A3CME8E2E2R7Q7']
merged_df = merged_df[~merged_df['worker_id'].isin(broken_restdb)].copy()
new_len = len(merged_df)
print(old_len, new_len)

344 336


In [119]:
# Drop some final extraneous vars
merged_df.drop(columns=['Answer.surveycode','Input.hit_id','Input.offer_amt',
                        'Input.offer_hash','Answer.hiddenhitid','restdb_wage',
                        'offer','RequesterAnnotation','AssignmentDurationInSeconds',
                        'AutoApprovalDelayInSeconds','NumberOfSimilarHITs',
                        'LifetimeInSeconds','AssignmentStatus','AutoApprovalTime',
                        'ApprovalTime','RejectionTime','RequesterFeedback',
                        'Answer.acceptOffer','Answer.surveyCode','Approve','Reject',
                        'Status','RecipientLastName','RecipientFirstName','RecipientEmail',
                        'ExternalReference','DistributionChannel','qualtrics_offer_amt',
                        'end_message','coin_flip','refuseoffer','qualtrics_offer',
                        'offer_case','_merge','mt_id','Kneywords',
                        # These are already in entered_wage, entered_hrs, etc.
                        'wage','hrs','controlhrs','sickleave','friends','commute',
                        'express','coworkers','suprespect','supfair'],
               inplace=True, errors='ignore')

In [120]:
# And a final rename for ease of use
final_rename = {'final_offer':'offer_amt',
                'WorkerId':'worker_id'}
merged_df.rename(columns=final_rename, inplace=True)

In [121]:
merged_df.to_csv("../results/results_clean.csv", index=False)

In [122]:
# The key vars: WorkerId, accepted_offer, and final_offer

In [123]:
merged_df['worker_id']

0      A26KX5PGHY4FZJ
1       AKQAI78JTXXC9
2        ADP4KXZZ72DP
3        AJ9IY4IHOGB8
4        AJ9IY4IHOGB8
            ...      
331    A2PE5OYTZCN8J9
332    A1G187YBG0DVMQ
333    A3TBG0S2IEBVHU
334     AWN8SISULPG1E
335    A1L82ZAF1DFQK1
Name: worker_id, Length: 336, dtype: object

In [124]:
merged_df[pd.isna(merged_df['worker_id'])]

,restdb_id,worker_id,restdb_timestamp,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,...,generated_jobtitle_23,generated_jobtitle_24,generated_jobtitle_25,generated_jobtitle_26,generated_jobtitle_27,generated_jobtitle_28,generated_jobtitle_29,generated_jobtitle_30,qualtrics_merge,offer_amt


In [125]:
merged_df['offer_amt'].value_counts()

$1.02    32
$0.95    32
$0.99    31
$1.00    31
$1.05    31
$0.50    31
$0.98    30
$0.90    30
$1.01    30
$1.50    29
$1.10    29
Name: offer_amt, dtype: int64

In [126]:
bad_bois = merged_df[pd.isna(merged_df['offer_amt'])].copy()

In [127]:
len(bad_bois)

0

In [128]:
bad_bois['worker_id']

Series([], Name: worker_id, dtype: object)

In [129]:
# These we also don't have 

In [130]:
bad_bois.to_csv("bad_bois.csv", index=False)

In [131]:
merged_df['accepted_offer'].value_counts()

True     151
False     60
Name: accepted_offer, dtype: int64

In [132]:
merged_df[pd.isna(merged_df['accepted_offer'])]

,restdb_id,worker_id,restdb_timestamp,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,...,generated_jobtitle_23,generated_jobtitle_24,generated_jobtitle_25,generated_jobtitle_26,generated_jobtitle_27,generated_jobtitle_28,generated_jobtitle_29,generated_jobtitle_30,qualtrics_merge,offer_amt
0,26.0,A26KX5PGHY4FZJ,1.625119e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,$0.99
1,25.0,AKQAI78JTXXC9,1.625118e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,$0.98
5,24.0,AH3E0I5Y9CMCO,1.625112e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,$0.95
6,23.0,A3IZUHHAA21KBM,1.625111e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,$0.90
7,27.0,A3O5RKGH6VB19C,1.625120e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,$1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,313.0,AWA9HQ5Q5T1YC,1.630205e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,$1.00
288,320.0,A15FD410ABX0Q2,1.630206e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,$0.90
290,322.0,A1P6OXEJ86HQRM,1.630206e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,$0.98
298,330.0,A3TP7NJZHR6WZ1,1.630208e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,$0.50


In [133]:
merged_df['response_case'].value_counts()

mt_submitter    194
mt_returner     125
no_restdb        17
Name: response_case, dtype: int64

In [134]:
len(merged_df)

336

In [135]:
merged_df.to_csv("../results/results_clean.csv", index=False)

In [136]:
col_names = merged_df.columns.values

In [148]:
col_names

array(['restdb_id', 'worker_id', 'restdb_timestamp', 'HITId', 'HITTypeId',
       'Title', 'Description', 'Keywords', 'Reward', 'CreationTime',
       'MaxAssignments', 'Expiration', 'AssignmentId', 'AcceptTime',
       'SubmitTime', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Answer.age',
       'Answer.onlinehrs', 'Answer.reason', 'source', 'accepted_offer',
       'mturk_code', 'rdb_mturk_merge', 'response_case', 'StartDate',
       'EndDate', 'IPAddress', 'Progress', 'Duration (in seconds)',
       'Finished', 'RecordedDate', 'ResponseId', 'LocationLatitude',
       'LocationLongitude', 'UserLanguage', 'state', 'race',
       'race_6_TEXT', 'gender', 'gender_4_TEXT', 'education',
       'currently_employed', 'fulltime', 'most_recent_job',
       'most_recent_fulltime', 'weeks_unemployment', 'how_easy',
       'lowest_wage', 'savings', 'jobtitle', 'Q11', 'Q22', 'Q12', 'Q90',
       'Q13', 'Q14', 'Q15', 'Q20', 'Q19', 'Q16', 

In [137]:
[c for c in col_names if "response" in c]

['response_case',
 'response1',
 'response2',
 'response3',
 'response4',
 'response5',
 'response6',
 'response7',
 'response8',
 'response9',
 'response10',
 'response11',
 'response12',
 'response13',
 'response14',
 'response15',
 'response16',
 'response17',
 'response18',
 'response19',
 'response20',
 'response21',
 'response22',
 'response23',
 'response24',
 'response25',
 'response26',
 'response27',
 'response28',
 'response29',
 'response30']

In [138]:
sum(~pd.isna(merged_df['response1']))

150

In [147]:
#with pd.option_context('max_rows',None):
#    display(merged_df['response1'])

In [145]:
merged_df.iloc[20]['response1']

nan

In [146]:
merged_df.iloc[20]

restdb_id                           40.0
worker_id                  AB3L4DBGHX7PG
restdb_timestamp         1625126106686.0
HITId                                NaN
HITTypeId                            NaN
                              ...       
generated_jobtitle_28                NaN
generated_jobtitle_29                NaN
generated_jobtitle_30                NaN
qualtrics_merge                left_only
offer_amt                          $1.02
Name: 20, Length: 540, dtype: object